In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn import metrics
from tabulate import tabulate
from datetime import timedelta
import matplotlib.pyplot as plt

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import UnivariateFeatureSelector
from pyspark.ml.feature import ChiSqSelector, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier as pyspark_DecisionTreeClassifier
from pyspark.sql.functions import col

In [ ]:
# Creating a new spark session
spark = SparkSession.builder.master("local[*]").appName("MLlib lab").getOrCreate()

In [ ]:
#sparkDF = spark.read.option("maxColumns", 22285).parquet("/mnt/the-data-transformers/spark_output/preprocessed_data_final2.pqt")
#sparkDF = spark.read.option("maxColumns", 22285).csv("/mnt/the-data-transformers/Leukemia_GSE9476_test.csv")
#sparkDF = spark.read.option("maxColumns", 20532).csv("/mnt/the-data-transformers/gene_encoded.csv", header='true')
sparkDF = spark.read.csv("/mnt/the-data-transformers/toxicity.csv", header='true')
"""
1 PRAD
2 LUAD
3 BRCA
4 KIRC
5 COAD
"""

Out[18]: '\n1 PRAD\n2 LUAD\n3 BRCA\n4 KIRC\n5 COAD\n'

In [ ]:
sparkDF = sparkDF.select(*(col(c).cast("float").alias(c) for c in sparkDF.columns))
sparkDF.head()

Out[24]: Row(Column1=0.0, Column2=0.0, Column3=0.0, Column4=0.0, Column5=0.0, Column6=0.0, Column7=0.0, Column8=0.0, Column9=0.0, Column10=0.0, Column11=1.0, Column12=0.0, Column13=0.0, Column14=0.0, Column15=0.0, Column16=0.0, Column17=1.0, Column18=0.0, Column19=0.0, Column20=0.0, Column21=0.0, Column22=0.0, Column23=0.0, Column24=0.0, Column25=0.0, Column26=0.0, Column27=0.0, Column28=0.0, Column29=0.0, Column30=0.0, Column31=0.0, Column32=0.0, Column33=0.0, Column34=0.0, Column35=0.0, Column36=0.0, Column37=0.0, Column38=0.0, Column39=0.0, Column40=0.0, Column41=1.0, Column42=0.0, Column43=0.0, Column44=0.0, Column45=0.0, Column46=0.0, Column47=0.0, Column48=0.0, Column49=0.0, Column50=0.0, Column51=0.0, Column52=0.0, Column53=0.0, Column54=0.0, Column55=0.0, Column56=0.0, Column57=0.0, Column58=0.0, Column59=0.0, Column60=0.0, Column61=0.0, Column62=0.0, Column63=0.0, Column64=0.0, Column65=0.0, Column66=0.0, Column67=0.0, Column68=0.0, Column69=0.0, Column70=0.0, Column71=0.0, Column72=0.0, Column73=0.0, Column74=0.0, Column75=0.0, Column76=0.0, Column77=0.0, Column78=0.0, Column79=1.0, Column80=0.0, Column81=0.0, Column82=0.0, Column83=1.0, Column84=0.0, Column85=0.0, Column86=0.0, Column87=0.0, Column88=0.0, Column89=0.0, Column90=0.0, Column91=0.0, Column92=0.0, Column93=1.0, Column94=0.0, Column95=0.0, Column96=0.0, Column97=0.0, Column98=0.0, Column99=0.0, Column100=0.0, Column101=0.0, Column102=0.0, Column103=0.0, Column104=0.0, Column105=0.0, Column106=0.0, Column107=0.0, Column108=0.0, Column109=0.0, Column110=0.0, Column111=0.0, Column112=0.0, Column113=0.0, Column114=0.0, Column115=0.0, Column116=0.0, Column117=0.0, Column118=0.0, Column119=0.0, Column120=0.0, Column121=0.0, Column122=0.0, Column123=0.0, Column124=0.0, Column125=0.0, Column126=0.0, Column127=0.0, Column128=0.0, Column129=0.0, Column130=0.0, Column131=0.0, Column132=0.0, Column133=0.0, Column134=0.0, Column135=0.0, Column136=0.0, Column137=0.0, Column138=0.0, Column139=0.0, Column140=0.0, Column141=0.0, Column142=0.0, Column143=0.0, Column144=0.0, Column145=0.0, Column146=0.0, Column147=0.0, Column148=0.0, Column149=0.0, Column150=0.0, Column151=0.0, Column152=0.0, Column153=0.0, Column154=0.0, Column155=0.0, Column156=0.0, Column157=0.0, Column158=0.0, Column159=0.0, Column160=0.0, Column161=0.0, Column162=0.0, Column163=0.0, Column164=0.0, Column165=0.0, Column166=0.0, Column167=0.0, Column168=0.0, Column169=0.0, Column170=0.0, Column171=0.0, Column172=0.0, Column173=0.0, Column174=0.0, Column175=0.0, Column176=0.0, Column177=0.0, Column178=0.0, Column179=1.0, Column180=1.0, Column181=1.0, Column182=0.0, Column183=0.0, Column184=0.0, Column185=0.0, Column186=1.0, Column187=0.0, Column188=0.0, Column189=0.0, Column190=0.0, Column191=0.0, Column192=0.0, Column193=0.0, Column194=0.0, Column195=1.0, Column196=0.0, Column197=0.0, Column198=0.0, Column199=0.0, Column200=0.0, Column201=0.0, Column202=0.0, Column203=0.0, Column204=0.0, Column205=0.0, Column206=0.0, Column207=0.0, Column208=0.0, Column209=0.0, Column210=0.0, Column211=0.0, Column212=0.0, Column213=0.0, Column214=0.0, Column215=0.0, Column216=0.0, Column217=0.0, Column218=0.0, Column219=0.0, Column220=0.0, Column221=0.0, Column222=0.0, Column223=1.0, Column224=0.0, Column225=0.0, Column226=1.0, Column227=0.0, Column228=0.0, Column229=0.0, Column230=0.0, Column231=0.0, Column232=0.0, Column233=0.0, Column234=0.0, Column235=0.0, Column236=0.0, Column237=0.0, Column238=0.0, Column239=0.0, Column240=0.0, Column241=0.0, Column242=0.0, Column243=0.0, Column244=0.0, Column245=0.0, Column246=0.0, Column247=0.0, Column248=0.0, Column249=0.0, Column250=0.0, Column251=0.0, Column252=0.0, Column253=0.0, Column254=0.0, Column255=0.0, Column256=0.0, Column257=0.0, Column258=0.0, Column259=0.0, Column260=0.0, Column261=0.0, Column262=0.0, Column263=0.0, Column264=0.0, Column265=0.0, Column266=0.0, Column267=0.0, Column268=1.0, Column269=0.0, Column270=0.0, Column271=0.0, Column272=0.0, Column273=0.0, 

In [ ]:
sparkDF.count()

Out[20]: 8992

In [ ]:
len(sparkDF.columns)

Out[21]: 1025

In [ ]:
'''
Function: spark_UnivariateFeatureSelector
INPUT:
------
i: Number of Features to select
_vector_sparkDF: DataFrame from which the Features will be selected

OUTPUT:
-------
1. The feature importances returned by the newly trained model
'''
def spark_UnivariateFeatureSelector(i, _vector_sparkDF):
   
  # Selecting the best i features from the entire dataset
  selector = UnivariateFeatureSelector(
    featuresCol="features", 
    outputCol="selectedFeatures", 
    labelCol="type", 
    selectionMode="numTopFeatures"
  )
  selector.setFeatureType("continuous").setLabelType("categorical").setSelectionThreshold(i)
  result = selector.fit(_vector_sparkDF).transform(_vector_sparkDF)

  # Using pyspark DecisionTreeClassifier to define and fit the ML model
  dt = pyspark_DecisionTreeClassifier(labelCol="type", featuresCol="selectedFeatures")
  model = dt.fit(result)
  
  return model.featureImportances 


In [ ]:
'''
Function: get_important_features
INPUT:
------
num: the number of features present in a chunk/subset
i: the chunk number

OUTPUT:
-------
1. The features selected by the UnivariateFeatureSelector
'''
def get_important_features(num, i):
  num_columns = num
  num_features = num
  
  # get chunk using indexing
  print("range, lower: "+str(1+(i*num_columns))+ " upper:"+str(num_columns*(i+1)))
  chunk =  sparkDF.select(*sparkDF.columns[1+(i*num_columns):num_columns*(i+1)], sparkDF['type']).distinct()
  vecAssembler = VectorAssembler(inputCols=chunk.columns, outputCol="features")
  newDF = vecAssembler.transform(chunk)
  selected_features = spark_UnivariateFeatureSelector(num_features, newDF)

  return selected_features


'''
Function: remove_features
INPUT:
------
sparkDF: the spark DataFrame consisting of the dataset

OUTPUT:
-------
1. The new spark DataFrame that only contains the most important subset of features
'''
def remove_features(sparkDF):
  print("removing features")
  columns = 100 # 2000
  iterations = 10 # 11
  toDrop = []
  
  # loop through chunks
  for i in range(0, iterations):
    print("chunk: "+str(i+1))

    # set num columns to amount left if below set columns
    if (len(sparkDF.columns) < columns):
      print("updating remaining columns")
      columns = len(sparkDF.columns)
      
    # get important features
    important_feature_vectors = get_important_features(columns, i) 
    vector = important_feature_vectors.toArray()
    index  = columns*i
    
    # loop through all colums and and add low importance ones to array
    for c in range(0, columns):
      if (vector[c] == 0): # if unimportant
        tmp_index = index + c
        if (sparkDF.columns[tmp_index] and sparkDF.columns[tmp_index] != "type"): # if index is correct
          toDrop.append(sparkDF.columns[tmp_index])
        
  # drop low importance columns
  sparkDF = sparkDF.drop(*toDrop)
  print("remaining columns: "+str(len(sparkDF.columns)))

  return sparkDF

sparkDF = remove_features(sparkDF)

removing features
chunk: 1
range, lower: 1 upper:100
chunk: 2
range, lower: 101 upper:200
chunk: 3
range, lower: 201 upper:300
chunk: 4
range, lower: 301 upper:400
chunk: 5
range, lower: 401 upper:500
chunk: 6
range, lower: 501 upper:600
chunk: 7
range, lower: 601 upper:700
chunk: 8
range, lower: 701 upper:800
chunk: 9
range, lower: 801 upper:900
chunk: 10
range, lower: 901 upper:1000
remaining columns: 35

In [ ]:
len(sparkDF.columns)

In [ ]:
# Make predictions
selector = UnivariateFeatureSelector(
  featuresCol="features", 
  outputCol="selectedFeatures", 
  labelCol="type", 
  selectionMode="numTopFeatures"
)

vecAssembler = VectorAssembler(inputCols=sparkDF.columns, outputCol="features")
sparkDF = vecAssembler.transform(sparkDF)
  
selector.setFeatureType("continuous").setLabelType("categorical") #.setSelectionThreshold(i)
result = selector.fit(sparkDF).transform(sparkDF)

# Splitting the data into training & testing
(train, test) = result.randomSplit([0.8, 0.2])

# Using pyspark DecisionTreeClassifier to define and fit the ML model
dt = pyspark_DecisionTreeClassifier(labelCol="type", featuresCol="selectedFeatures")
model = dt.fit(train)
predictions = model.transform(test)

# Evaluating the predictions
evaluator = MulticlassClassificationEvaluator(
  labelCol="type", 
  predictionCol="prediction", 
  metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print("Test accuracy = "+str(accuracy))


Test accuracy = 1.0